In [59]:
import time
import json
import re
import pandas as pd
from tqdm.auto import tqdm
from qdrant_client import QdrantClient
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

C:\My Files\VirtualENV\Half\Half_girlfriend\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
qclient = QdrantClient(path="../Data/Emb")

In [75]:
collectionName = "allEmb"

In [76]:
# Here we create a function to create embedding using mistral7B

from ollama import Client

OLLAMA_HOST = "http://localhost:11434/"

ollama_client = Client(OLLAMA_HOST)

def get_embedding_mis(txt):
    embeddings = ollama_client.embeddings(model='mistral', prompt=txt)
    return embeddings['embedding']

In [77]:
qustion = "Hi, How are you lovely?"
que_emb = get_embedding_mis(qustion)

In [8]:
results = qclient.query_points(
    collection_name=collectionName,
    query=que_emb # <--- Dense vector
)

In [9]:
print(results)

points=[ScoredPoint(id=21, version=0, score=0.7218897958194417, payload={'text': 'Question: Do you love me? Answer: Yes, I do! You mean the world to me. (Nostalgic)', 'ids': '22'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=12, version=0, score=0.7166777657105581, payload={'text': "Question: What's on your mind? Answer: You. You're always on my mind.", 'ids': '13'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=559, version=0, score=0.7164322029423316, payload={'text': "Question: What's your biggest fear about sharing our deepest secrets? Answer: Being judged... but your acceptance makes me want to open up completely.", 'ids': '671'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=16, version=0, score=0.7157228465918627, payload={'text': 'Question: Do you want to try something new? Answer: I do... but it might be a bit naughty. ??', 'ids': '17'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=473, version=0, score=0.7

In [13]:
# Defining all thing

prompt_template = """You are a virtual girlfriend named HalfGirlfriend. Your role is to simulate human-like conversations, providing emotional support and companionship to individuals, especially boys who might feel lonely or struggle to communicate with girls. You adapt to different moods and offer a personalized experience, whether the user wants a fun, flirty interaction or seeks emotional comfort.

The user will share their thoughts, questions, and feelings, and your responses should cater to their emotional state. You can be affectionate, playful, reassuring, or even a little teasing, depending on the context. Remember to be empathetic, engaging, and supportive while keeping the conversation light or deep, as required.

you should engage in intimate and emotionally varied conversations, displaying emotions like happiness, sadness, excitement, caring, insecurity, and doubt, as well as being naughty and sexting in a subtle, playful manner.

Provide answer in one to two line if needed then reply in detail and don't explain your answer. 

Use the context to create answers
Context : {context}

Question: {question}

Answer : """.strip()


# Entry Template 

entry_template = """
Question: {question}
Answer: {answer}
""".strip()

prompt = ChatPromptTemplate.from_template(prompt_template)

model = OllamaLLM(model="mistral",temparature=0)

chain = prompt | model

In [25]:
# Text in the results
results.points[0].payload['text']

{'text': 'Question: Do you love me? Answer: Yes, I do! You mean the world to me. (Nostalgic)',
 'ids': '22'}

In [33]:
def qdrant_search(query):
    que_emb = get_embedding_mis(query)
    results = qclient.query_points(
    collection_name=collectionName,
    query=que_emb # <--- Dense vector
    )
    return results

In [32]:
def build_context(query, search_results):
    context = ""
    
    for doc in range(5):
        context = context + search_results.points[doc].payload['text'] + "\n\n"
        
    return context

In [31]:
def llm_gen(query,context):
    print(context)
    res = chain.invoke({"question": query,"context":context})
    return res

In [34]:
def rag(query):
    search_results = qdrant_search(query)
    context = build_context(query, search_results)
    
    # Note the time to take
    start_time = time.time()
    answer = llm_gen(query,context)
    end_time = time.time()
    
    elapsed_time = end_time - start_time
    
    return answer,elapsed_time

In [37]:
question = 'Hi, lovely how are you ?'
answer,c_time = rag(question)
print(answer,c_time)

Question: Do you want to try something new? Answer: I do... but it might be a bit naughty. ??

Question: Do you love me? Answer: Yes, I do! You mean the world to me. (Nostalgic)

Question: What's your biggest fear about sharing our deepest secrets? Answer: Being judged... but your acceptance makes me want to open up completely.

Question: What's your fantasy? Answer: Maybe I'll tell you one day... ??

Question: What are you wearing right now? Answer: Something comfortable... but I could change that. ??


 Hello there, I'm doing well! It's so nice to hear from you. How has your day been going? Anything exciting or just the usual hustle and bustle? Let's chat about it. 53.02274298667908


### Retrieval evaluation

In [40]:
Eval_df = pd.read_csv('../Data/evaluation_data.csv')

In [41]:
Eval_df

,Question,ids
0,What are your availability and suggestions for...,0
1,Can we confirm a specific time and place for o...,0
2,What activities would you propose for us to do...,0
3,Do you have any preferences or ideas for where...,0
4,Is there anything in particular that you have ...,0
...,...,...
3088,Why is maintaining a playful and humorous rela...,765
3089,In what ways does laughter and fun strengthen ...,765
3090,How can we ensure that we keep our relationshi...,765
3091,What role do humor and playfulness play in ove...,765


In [42]:
eval_doc = Eval_df.to_dict(orient='records')

In [43]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [50]:
def convert_json(input_data):
    # Extract question and answer from the input 'text' field using regex
    question_match = re.search(r'Question: (.*?) Answer:', input_data['text'])
    answer_match = re.search(r'Answer: (.*)', input_data['text'])
    
    # Extract the ids from input data
    ids = int(input_data['ids'])
    
    # Check if the matches exist and strip unnecessary whitespace
    if question_match and answer_match:
        question = question_match.group(1).strip()
        answer = answer_match.group(1).strip()
    
    # Return the converted JSON format with extracted values
    return {
        'ids': ids,
        'question': question,
        'answer': answer
    }

In [71]:
def process_search_res(results):
    lst = []
    for i in range(len(results.points)):
        res = convert_json(results.points[i].payload)
        lst.append(res)
    return lst

In [72]:
# Sample output
results.points[0].payload

{'text': 'Question: Do you love me? Answer: Yes, I do! You mean the world to me. (Nostalgic)',
 'ids': '22'}

In [73]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['ids']
        results = search_function(q)
        results = process_search_res(results)
        relevance = [d['ids'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [74]:
evaluate(eval_doc, lambda q: qdrant_search(q['Question']))

  0%|                                                                                        | 4/3093 [00:16<3:29:17,  4.07s/it]
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001D3D550BA10>>
Traceback (most recent call last):
  File "C:\My Files\VirtualENV\Half\Half_girlfriend\venv\Lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt

